<a href="https://colab.research.google.com/github/dilasko/barbot/blob/main/barbot1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install pandas gradio google-genai

In [ ]:
import pandas as pd  # Veri işleme ve CSV dosyasını okuma için Pandas kütüphanesi.
import gradio as gr   # Web arayüzü (UI) oluşturmak ve uygulamayı sunmak için Gradio kütüphanesi.
import random         # Rastgele tarif seçimi gibi işlemler için kullanılır.
import os             # Ortam değişkenlerini okuma (API Anahtarı) ve dosya işlemleri için.
import re             # Düzenli ifadeler (Regex) ile malzeme aramayı hassaslaştırmak için.
import time           # API çeviri hatalarında bekleme (retry delay) süresi için.
# Gemini API kütüphaneleri
from google import genai           # Gemini (Google AI) servisine erişim için ana kütüphane.
from google.genai.errors import APIError # API çağrılarından kaynaklanan özel hataları yakalamak için.

# ----------------------------------------------------------------------
# 1. VERİ YÜKLEME VE API İSTEMCİSİ AYARLARI
# ----------------------------------------------------------------------

# Veri setini yükler.
try:
    df = pd.read_csv('final_cocktails.csv')  # Veri setini okur.
    # Arama hassasiyetini artırmak için, malzemeler sütunundaki tüm metni küçük harfe çevirir.
    df['ingredients_lower'] = df['ingredients'].astype(str).str.lower()
except FileNotFoundError:
    # CSV dosyası bulunamazsa kullanıcıya hata mesajı gösterir.
    print("HATA: 'final_cocktails.csv' dosyası bulunamadı. Lütfen dosyayı Colab'e yüklediğinizden emin olun.")
    df = pd.DataFrame()  # Hata durumunda boş bir DataFrame ile devam eder.

# Gemini API istemcisini başlatma
GEMINI_ENABLED = False  # Gemini'nin etkin olup olmadığını belirten bayrak.
client = None           # Gemini Client nesnesini tutacak değişken.
try:
    # 📌 BURAYI DÜZENLEYİN: API Anahtarınızı tırnak işaretleri içine yazın
    API_KEY_SABİT = 'SİZİN_API_ANAHTARINIZI_BURAYA_YAZIN' # Sabit anahtar tanımı (önerilen yer).

    # Ortam değişkenini kontrol eder (Colab Sırlar veya Yerel 'GEMINI_API_KEY').
    api_key_env = os.environ.get('GEMINI_API_KEY')

    # Anahtar belirleme mantığı: Sabit anahtar önceliklidir.
    if API_KEY_SABİT and API_KEY_SABİT != 'SİZİN_API_ANAHTARINIZI_BURAYA_YAZIN':
        api_key = API_KEY_SABİT
    elif api_key_env:
        api_key = api_key_env
    else:
        api_key = None

    if api_key:
        client = genai.Client(api_key=api_key)  # İstemciyi anahtar ile başlatır.
        GEMINI_ENABLED = True                  # Bayrağı etkinleştirir.
        print("✅ Gemini API istemcisi başarıyla başlatıldı. Çeviri ve Görsel oluşturma aktif.")
    else:
        print("❌ Gemini API Anahtarı bulunamadı. Çeviri ve Görsel oluşturma devre dışı.")

except Exception as e:
    print(f"❌ Gemini API istemcisi başlatılamadı: {e}")

# ----------------------------------------------------------------------
# 2. GEMINI ÇEVİRİ VE GÖRSEL FONKSİYONLARI
# ----------------------------------------------------------------------

def safe_translate(prompt, model='gemini-2.5-flash'):
    """Genel, güvenli çeviri işlevi. Çeviri isteğini API'ye gönderir."""
    if not GEMINI_ENABLED:
        return None
    try:
        response = client.models.generate_content(
            model=model,
            contents=prompt
        )
        return response.text.strip()
    except Exception as e:
        # API hatası veya bağlantı sorunu durumunda None döndürür.
        return None

def translate_item(item_en, context_prompt):
    """Kısa, tek bir terimi (Kategori, Bardak Tipi vb.) çevirir."""
    result = safe_translate(
        prompt=f"Bu İngilizce terimi Türkçe'ye çevir: '{item_en}'. Çeviri yaparken sadece çevrilmiş terimi döndür. Bağlam: {context_prompt}"
    )
    # Çeviri başarısız olursa orijinal İngilizce terimi döndürür.
    return result if result else item_en

def translate_instructions(text_en, max_retries=3):
    """Kokteyl talimatlarını profesyonel bir dille çevirir. API hatasına karşı yeniden deneme mekanizması içerir."""
    if not GEMINI_ENABLED:
        return f"**(Çeviri yapılamadı - Orijinal Metin)**:\n{text_en}"

    for attempt in range(max_retries):
        # API sınırını aşmamak ve daha hızlı yanıt almak için metni kısaltır.
        short_text = text_en[:1000]

        result = safe_translate(
            prompt=f"Bu kokteyl yapım talimatını profesyonel bir barmen edasıyla ve doğal bir dille Türkçe'ye çevir: {short_text}"
        )

        # Başarılı bir sonuç gelirse döngüden çıkar.
        if result and "Çeviri yapılamadı" not in result:
            return result

        # Yeniden deneme yapılması gerekiyorsa bekler.
        if attempt < max_retries - 1:
            print(f"⚠️ Talimat çevirisi başarısız oldu (Deneme {attempt + 1}). 2 saniye bekleniyor...")
            time.sleep(2)

    # Tüm denemeler başarısız olursa hata mesajını döndürür.
    return f"**(Çeviri yapılamadı - Orijinal Metin)**:\n{text_en}"


def translate_ingredient_list(list_en):
    """Malzeme listesinin tamamını çevirir (tüm ölçüleri koruyarak)."""
    result = safe_translate(
        prompt=f"Aşağıdaki kokteyl malzeme listesini ve ölçülerini tam ve doğru Türkçe'ye çevir. Formatı (madde işaretleri) aynen koru. Sadece çevrilmiş listeyi döndür:\n{list_en}"
    )
    return result if result else f"**(Çeviri yapılamadı - Orijinal Liste)**:\n{list_en}"

def safe_translate_to_en(text_tr, model='gemini-2.5-flash'):
    """Kullanıcı girdisini arama yapabilmek için İngilizce'ye çevirir."""
    if not GEMINI_ENABLED:
        return text_tr

    prompt = f"Bu malzeme listesini/ifadeyi tamamen İngilizce'ye çevir. Sadece İngilizce çeviriyi döndür. İfade: '{text_tr}'"

    try:
        response = client.models.generate_content(
            model=model,
            contents=prompt
        )
        translated_text = response.text.strip().lower()
        if len(translated_text) > 0:
            return translated_text
        return text_tr
    except Exception as e:
        return text_tr

def generate_cocktail_image(cocktail_name, instructions_tr):
    """Görsel URL'si bulunamazsa, tarife özel görsel oluşturur (Imagen modeli kullanılır)."""
    if not GEMINI_ENABLED:
        return None

    # Prompt (komut) oluşturulur.
    prompt = (
        f"Profesyonel fotoğrafçılık kalitesinde, '{cocktail_name}' kokteylinin hazırlanışını veya "
        f"servis edildiği anı yansıtan, iştah açıcı ve şık bir fotoğrafını oluştur. "
        f"Tarif detayları: {instructions_tr[:200]}..."
    )

    try:
        print(f"🔄 Gemini ile görsel oluşturuluyor: {cocktail_name}...")
        result = client.models.generate_images(
            model='imagen-3.0-generate-002', # Yüksek kaliteli görsel oluşturma modeli.
            prompt=prompt,
            config=dict(
                number_of_images=1,
                output_mime_type="image/jpeg",
                aspect_ratio="1:1"
            )
        )

        if result.generated_images:
            image_data = result.generated_images[0].image.image_bytes
            # Gradio'nun gösterebilmesi için görseli geçici bir dosyaya kaydeder.
            temp_filename = f"temp_cocktail_image_{random.randint(1000, 9999)}.jpg"
            with open(temp_filename, 'wb') as f:
                f.write(image_data)
            return temp_filename

    except APIError as e:
        print(f"⚠️ Görsel oluşturma API hatası: {e}")
        return None
    except Exception as e:
        print(f"⚠️ Görsel oluşturma hatası: {e}")
        return None

    return None

# ----------------------------------------------------------------------
# 3. ANA CHATBOT FONKSİYONU
# ----------------------------------------------------------------------

def kokteyl_tarifi_al(malzeme_girisi):
    """
    Kullanıcının girdiği malzemeleri işler, tarif bulur, çevirir ve sonucu döndürür.
    """
    if df.empty:
        return "Veri seti yüklenemedi. Lütfen dosyanın adını kontrol edin.", None

    # Girdiyi İngilizce'ye çevirir (Türkçe arama desteği için).
    if GEMINI_ENABLED:
        processed_input = safe_translate_to_en(malzeme_girisi)
    else:
        processed_input = malzeme_girisi

    # Girdiyi virgülle ayırır ve temizler.
    search_terms = [term.strip().lower() for term in processed_input.split(',') if term.strip()]

    if not search_terms:
        output_text = "⚠️ Lütfen bir malzeme ismi veya virgülle ayrılmış birden fazla malzeme giriniz."
        return output_text, None

    # FİLTRELEME MANTIĞI: Tek/Çok Girdi Kontrolü
    filter_condition = pd.Series([True] * len(df))
    is_single_ingredient = len(search_terms) == 1 # Tek malzeme olup olmadığını kontrol eder.

    for term in search_terms:
        if is_single_ingredient:
            # TEK MALZEME: Esnek arama (örn: 'chocolate' -> 'chocolate liqueur').
            filter_condition = filter_condition & df['ingredients_lower'].str.contains(
                term,
                na=False,
                regex=False # Kelime sınırını dikkate almaz.
            )
        else:
            # ÇOK MALZEME: Hassas arama (yanlış eşleşmeleri engellemek için).
            regex_pattern = r'\b' + re.escape(term) + r'\b' # Tam kelime sınırını zorlar.
            filter_condition = filter_condition & df['ingredients_lower'].str.contains(
                regex_pattern,
                na=False,
                regex=True
            )

    filtered_df = df[filter_condition]

    if filtered_df.empty:
        terimler = ", ".join([f"'{t}'" for t in search_terms])

        if is_single_ingredient:
             output_text = f"❌ Girdiğiniz **'{terimler}'** malzemesini (veya türevlerini) içeren bir kokteyl tarifi bulunamadı. Başka malzemeler deneyin."
        else:
             output_text = f"❌ Girdiğiniz **TÜM** malzemeleri (**{terimler}**) içeren bir kokteyl tarifi bulunamadı. Lütfen daha az veya farklı malzemeler deneyin."

        return output_text, None

    # Bulunan sonuçlar arasından rastgele bir tarif seçilir.
    selected_recipe = filtered_df.sample(n=1).iloc[0]

    # Seçilen tarifin detayları çekilir.
    name = selected_recipe['name']
    alcoholic_en = selected_recipe['alcoholic']
    category_en = selected_recipe['category']
    glass_en = selected_recipe['glassType']
    instructions_en = selected_recipe['instructions']
    ingredients_en = selected_recipe['ingredients'].strip("[]").replace("'", "")
    measures_en = selected_recipe['ingredientMeasures'].strip("[]").replace("'", "")
    thumbnail_url = selected_recipe['drinkThumbnail']

    # TALİMAT METNİ TEMİZLİĞİ: Uzun, genel uyarı metinlerini tariften ayırır.
    if "When adding spirits like vodka" in instructions_en:
        instructions_clean = instructions_en.split("When adding spirits like vodka")[0].strip()
    elif "Here are some general reminders" in instructions_en:
         instructions_clean = instructions_en.split("Here are some general reminders")[0].strip()
    elif "Don't Drink and Drive" in instructions_en:
        instructions_clean = instructions_en.split("Don't Drink and Drive")[0].strip()
    else:
        instructions_clean = instructions_en

    instructions_to_translate = instructions_clean


    # 4. TÜM ALANLARI ÇEVİRME
    if alcoholic_en == 'Alcoholic':
        alcoholic_tr = "Evet (Alkollü)"
    elif alcoholic_en == 'Non Alcoholic':
        alcoholic_tr = "Hayır (Alkolsüz)"
    else:
        alcoholic_tr = translate_item(alcoholic_en, "Alkollü olup olmadığını belirten bir durum.")

    category_tr = translate_item(category_en, "Kokteylin ait olduğu içecek kategorisi.")
    glass_tr = translate_item(glass_en, "Kokteylin servis edildiği bardak tipi.")

    # Malzeme listesi ve ölçüleri birleştirilip çevrilir.
    try:
        ingredient_list_en = [
            f"• {m.strip()} {i.strip()}"
            for m, i in zip(measures_en.split(','), ingredients_en.split(','))
        ]
        ingredients_formatted_en = "\n".join(ingredient_list_en)
    except Exception:
        ingredients_formatted_en = f"Malzemeler: {ingredients_en}\nÖlçüler: {measures_en}"

    ingredients_formatted_tr = translate_ingredient_list(ingredients_formatted_en)
    instructions_tr = translate_instructions(instructions_to_translate)

    # 5. GÖRSEL İŞLEME VE OLUŞTURMA
    image_path = None
    image_source_text = ""

    if pd.notna(thumbnail_url) and thumbnail_url:
         image_path = thumbnail_url
    elif GEMINI_ENABLED:
        image_path = generate_cocktail_image(name, instructions_tr) # Görsel oluşturmayı dener.
        if image_path:
             image_source_text = "\n\n(Bu görsel, Gemini yapay zekası tarafından tarifinize özel olarak oluşturulmuştur.)"

    # SONUCU TÜRKÇE OLARAK BİÇİMLENDİRME
    output_text = f"""
    ✨ **Kokteyl Adı:** {name}
    ---
    🍸 **Kategori:** {category_tr}
    🧊 **Bardak Tipi:** {glass_tr}
    🥂 **Alkollü:** {alcoholic_tr}
    ---
    **🧪 Malzemeler ve Ölçüler:**
    {ingredients_formatted_tr}
    ---
    **📜 Yapılış (Türkçe Talimatlar):**
    {instructions_tr}
    {image_source_text}
    """

    # Biçimlendirilmiş metin ve görsel yolunu Gradio'ya döndürür.
    return output_text.strip(), image_path

# ----------------------------------------------------------------------
# 4. GRADIO BLOCKS ARAYÜZÜ
# ----------------------------------------------------------------------

with gr.Blocks(title="🍸 Kokteyl Tarif Chatbot'u B-A-R*B*O*T") as demo:
    gr.Markdown(
        """
        # 🍸 Kokteyl Tarif Chatbot'u B-A-R*BOT
        Elindeki bir veya birden fazla malzemeyi **Türkçe veya İngilizce** virgülle ayırarak gir, sistem sana **girdiğin tüm malzemeleri** içeren bir kokteyl tarifi sunsun!

        *İpucu: Görsel bulunamazsa, Gemini sizin için özel bir görsel oluşturacaktır! (Gemini API Anahtarı kuruluysa)*
        """
    )

    # Giriş elemanları (Yatay satırda)
    with gr.Row():
        ingredient_input = gr.Textbox(
            lines=1,
            placeholder="Virgülle ayırarak birden fazla malzeme girin (Örn: Cin, Limon veya Gin, Lime)",
            label="Hangi malzemelerle kokteyl tarifi istersiniz?"
        )
        submit_btn = gr.Button("Tarifi Bul", variant="primary", scale=0)

    # Çıktı elemanları (Yatay satırda)
    with gr.Row():
        recipe_output = gr.Textbox(
            label="Kokteyl Tarifi",
            scale=2, # Çıktı alanının görselden iki kat daha geniş olmasını sağlar.
            lines=25
        )

        image_output = gr.Image(
            label="Kokteyl Görseli",
            type="pil",
            scale=1
        )

    # Olay dinleyicisi: Butona tıklandığında `kokteyl_tarifi_al` fonksiyonunu çalıştırır.
    submit_btn.click(
        fn=kokteyl_tarifi_al,
        inputs=ingredient_input,
        outputs=[recipe_output, image_output]
    )

    # Örnek girdiler (kullanıcının denemesini kolaylaştırır).
    gr.Examples(
        examples=[
            ["Çikolata"], # Tek malzeme (Esnek arama)
            ["Cin, Limon"], # Çok malzeme (Hassas arama)
            ["Rom, Limon"],
            ["Viski, Şeker, Angostura"],
        ],
        inputs=ingredient_input
    )

# Colab'de çalıştırmak için launch() metodunu kullanır.
demo.launch(share=True)